## 特征工程

In [1]:
# 工具导入
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import gc
from collections import Counter
import copy

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# 数据读取
test_data = pd.read_csv('dataset/data_format1/test_format1.csv')
train_data = pd.read_csv('dataset/data_format1/train_format1.csv')

user_info = pd.read_csv('dataset/data_format1/user_info_format1.csv')
user_log = pd.read_csv('dataset/data_format1/user_log_format1.csv')

In [3]:
# 对数据进行内存压缩
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    numerics = ['int16', 'int32','int64','float16','float32','float64']
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('decrease by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
num_rows = None
train_file = './dataset/data_format1/train_format1.csv'
test_file = './dataset/data_format1/test_format1.csv'

user_info_file = './dataset/data_format1/user_info_format1.csv'
user_log_file = './dataset/data_format1/user_log_format1.csv'

train_data = reduce_mem_usage(pd.read_csv(train_file, num_rows))
test_data = reduce_mem_usage(pd.read_csv(test_file, num_rows))
user_info = reduce_mem_usage(pd.read_csv(user_info_file, num_rows))
user_log = reduce_mem_usage(pd.read_csv(user_log_file, num_rows)) # 运行太慢

memory usage after optimization is: 1.74 MB
decrease by 70.8%
memory usage after optimization is: 3.49 MB
decrease by 41.7%
memory usage after optimization is: 3.24 MB
decrease by 66.7%


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260864 entries, 0 to 260863
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   user_id      260864 non-null  int64
 1   merchant_id  260864 non-null  int64
 2   label        260864 non-null  int64
dtypes: int64(3)
memory usage: 6.0 MB


In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261477 entries, 0 to 261476
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      261477 non-null  int64  
 1   merchant_id  261477 non-null  int64  
 2   prob         0 non-null       float64
dtypes: float64(1), int64(2)
memory usage: 6.0 MB


In [7]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    424170 non-null  int64  
 1   age_range  421953 non-null  float64
 2   gender     417734 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB


In [8]:
user_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   item_id      int64  
 2   cat_id       int64  
 3   seller_id    int64  
 4   brand_id     float64
 5   time_stamp   int64  
 6   action_type  int64  
dtypes: float64(1), int64(6)
memory usage: 2.9 GB


In [9]:
# 数据处理
all_data = train_data.append(test_data)
all_data = all_data.merge(user_info, on=['user_id'], how='left')
del train_data, test_data, user_info
gc.collect()

15

In [10]:
all_data.head()

,user_id,merchant_id,label,prob,age_range,gender
0,34176,3906,0.0,NaN,6.0,0.0
1,34176,121,0.0,NaN,6.0,0.0
2,34176,4356,1.0,NaN,6.0,0.0
3,34176,2217,0.0,NaN,6.0,0.0
4,230784,4818,0.0,NaN,0.0,0.0


In [11]:
# 按时间排序
user_log = user_log.sort_values(['user_id', 'time_stamp'])

In [12]:
# 对每个用户合并所有字段，合并字段为item_id
list_join_func = lambda x: " ".join([str(i) for i in x])

agg_dict = {
    'item_id': list_join_func,
    'cat_id': list_join_func,
    'seller_id': list_join_func,
    'brand_id': list_join_func,
    'time_stamp': list_join_func,
    'action_type': list_join_func,
}

rename_dict = {
    'item_id': 'item_path',
    'cat_id': 'cat_path',
    'seller_id': 'seller_path',
    'brand_id': 'brand_path',
    'time_stamp': 'time_stamp_path',
    'action_type': 'action_type_path',
}

def merge_list(df_ID, join_columns, df_data, agg_dict, rename_dict):
    df_data = df_data\
        .groupby(join_columns)\
        .agg(agg_dict)\
        .reset_index()\
        .rename(columns=rename_dict)
    df_ID = df_ID.merge(df_data, on=join_columns, how='left')
    return df_ID
'''
按照user_id分组，分别按照agg_dict聚合，
得到与该user_id有过交互的所有item_id cat_id ...保存在item_path,cat_path,...里
'''
all_data = merge_list(all_data, 'user_id', user_log, agg_dict, rename_dict)

In [13]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522341 entries, 0 to 522340
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           522341 non-null  int64  
 1   merchant_id       522341 non-null  int64  
 2   label             260864 non-null  float64
 3   prob              0 non-null       float64
 4   age_range         519763 non-null  float64
 5   gender            514796 non-null  float64
 6   item_path         522341 non-null  object 
 7   cat_path          522341 non-null  object 
 8   seller_path       522341 non-null  object 
 9   brand_path        522341 non-null  object 
 10  time_stamp_path   522341 non-null  object 
 11  action_type_path  522341 non-null  object 
dtypes: float64(4), int64(2), object(6)
memory usage: 51.8+ MB


In [14]:
all_data

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
1,34176,121,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
2,34176,4356,1.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
3,34176,2217,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
4,230784,4818,0.0,NaN,0.0,0.0,191923 191923 191923 191923 964906 229470 2294...,1023 1023 1023 1023 662 664 664 1544 664 662 6...,3545 3545 3545 3545 4566 2537 2537 2420 2537 4...,5860.0 5860.0 5860.0 5860.0 6322.0 6066.0 6066...,601 601 601 601 614 614 614 614 614 614 618 61...,0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 ...
...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,NaN,NaN,6.0,0.0,802791 977305 351177 122937 21972 863063 10903...,602 602 602 602 552 1271 1271 662 662 821 662 ...,2823 2823 2664 2664 1076 2946 2781 4949 2412 4...,1128.0 1128.0 8154.0 8154.0 3549.0 5559.0 3305...,511 511 512 512 512 516 516 521 521 521 521 52...,3 3 2 2 2 3 3 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 ...
522337,97919,2341,NaN,NaN,8.0,1.0,484765 128769 128769 995386 128769 645625 9953...,737 464 464 464 464 464 464 464 464 464 464 46...,4408 235 235 235 235 3416 235 235 235 235 235 ...,6967.0 2020.0 2020.0 2020.0 2020.0 6242.0 2020...,626 707 707 710 710 710 710 710 710 710 710 71...,2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 3 ...
522338,97919,3971,NaN,NaN,8.0,1.0,484765 128769 128769 995386 128769 645625 9953...,737 464 464 464 464 464 464 464 464 464 464 46...,4408 235 235 235 235 3416 235 235 235 235 235 ...,6967.0 2020.0 2020.0 2020.0 2020.0 6242.0 2020...,626 707 707 710 710 710 710 710 710 710 710 71...,2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 3 ...
522339,32639,3536,NaN,NaN,0.0,0.0,394570 394570 394570 28017 110194 314126 95836...,1413 1413 1413 812 1271 1271 1271 1198 1271 11...,1065 1065 1065 1506 38 1890 2280 4873 2280 487...,6374.0 4468.0 6374.0 4888.0 7010.0 5683.0 5372...,523 523 523 525 617 617 723 723 723 723 807 81...,0 2 0 0 0 0 0 0 0 2 0 0 0 0 2 0 2 0 0 3 0 0 0 ...


In [15]:
del user_log
gc.collect()

60

In [16]:
# 定义特征统计函数
def cnt_(x):
    try:
        return len(x.split(' '))
    except:
        return -1

def nunique(x):
    try:
        return len(set(x.split(' ')))
    except:
        return -1

def max_(x):
    try:
        return np.max([float(i) for i in x.split(' ')])
    except:
        return -1

def min_(x):
    try:
        return np.min([float(i) for i in x.split(' ')])
    except:
        return -1

def std_(x):
    try:
        return np.std([float(i) for i in x.split(' ')])
    except:
        return -1
# 定义统计数据中对topN数据的函数
def most_n(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][0]
    except:
        return -1
# 定义统计数据中对topN数据总数的函数
def most_n_cnt(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][1]
    except:
        return -1

In [17]:
# 调用定义的统计函数
'''
df_data: 数据
single_col：要进行统计的列名
name：统计完成新建的列名
'''
def user_cnt(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(cnt_)
    return df_data

def user_nunique(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(nunique)
    return df_data

def user_max(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(max_)
    return df_data

def user_min(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(min_)
    return df_data

def user_std(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(std_)
    return df_data

def user_most_n(df_data, single_col, name, n=1):
    func = lambda x: most_n(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data

def user_most_n_cnt(df_data, single_col, name, n=1):
    func = lambda x: most_n_cnt(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data

#### 3.3.6 提取统计特征

In [18]:
# 特征统计
# 店铺特征统计：统计与店铺特点有关的特征，如店铺、商品、品牌等
all_data_test = all_data.head(2000)
# 统计用户点击、浏览、加购、购买行为
# 总次数
all_data_test = user_cnt(all_data_test, 'seller_path', 'user_cnt')
# 不同店铺个数
all_data_test = user_nunique(all_data_test, 'seller_path', 'seller_nunique')
# 不同品类个数
all_data_test = user_nunique(all_data_test, 'cat_path', 'cat_nunique')
# 不同品牌个数
all_data_test = user_nunique(all_data_test, 'brand_path', 'brand_nunique')
# 不同商品个数
all_data_test = user_nunique(all_data_test, 'item_path', 'item_nunique')
# 活跃天数
all_data_test = user_nunique(all_data_test, 'time_stamp_path', 'time_stamp_nunique')
# 不同用户行为种数
all_data_test = user_nunique(all_data_test, 'action_type_path', 'action_type_nunique')
# 最晚时间
all_data_test = user_max(all_data_test, 'action_type_path', 'time_stamp_max')
# 最早时间
all_data_test = user_min(all_data_test, 'action_type_path', 'time_stamp_min')
# 活跃天数方差
all_data_test = user_std(all_data_test, 'action_type_path', 'time_stamp_std')
# 最早和最晚相差天数
all_data_test['time_stamp_range'] = all_data_test['time_stamp_max'] - all_data_test['time_stamp_min']
# 用户最喜欢的店铺
all_data_test = user_most_n(all_data_test, 'seller_path', 'seller_most_1', n=1)
# 最喜欢的类目
all_data_test = user_most_n(all_data_test, 'cat_path', 'cat_most_1', n=1)
# 最喜欢的品牌
all_data_test = user_most_n(all_data_test, 'brand_path', 'brand_most_1', n=1)
# 最常见的行为动作
all_data_test = user_most_n(all_data_test, 'action_type_path', 'action_type_most_1', n=1)
# ...
# 用户最喜欢的店铺 行为次数
all_data_test = user_most_n_cnt(all_data_test, 'seller_path', 'seller_most_1_cnt', n=1)
# 最喜欢的类目 行为次数
all_data_test = user_most_n_cnt(all_data_test, 'cat_path', 'cat_most_1_cnt', n=1)
# 最喜欢的品牌 行为次数
all_data_test = user_most_n_cnt(all_data_test, 'brand_path', 'brand_most_1_cnt', n=1)
# 最常见的行为动作 行为次数
all_data_test = user_most_n_cnt(all_data_test, 'action_type_path', 'action_type_most_1_cnt', n=1)

In [19]:
all_data_test.describe()

,user_id,merchant_id,label,prob,age_range,gender,user_cnt,seller_nunique,cat_nunique,brand_nunique,...,time_stamp_nunique,action_type_nunique,time_stamp_max,time_stamp_min,time_stamp_std,time_stamp_range,seller_most_1_cnt,cat_most_1_cnt,brand_most_1_cnt,action_type_most_1_cnt
count,2000.000000,2000.000000,2000.000000,0.0,1995.000000,1983.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.000000
mean,208239.475500,2533.319000,0.064500,NaN,3.135840,0.349975,140.082500,35.701500,23.987000,35.374000,...,17.845500,2.63700,2.577000,0.004000,0.743902,2.57300,29.575000,30.53200,29.844500,123.778000
std,122331.596829,1457.359826,0.245703,NaN,1.889127,0.530178,202.303623,35.853021,18.091365,35.298146,...,14.733505,0.54807,0.494159,0.089376,0.264871,0.50279,46.845711,39.37676,47.037364,186.462639
min,132.000000,10.000000,0.000000,NaN,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,...,2.000000,1.00000,2.000000,0.000000,0.000000,0.00000,1.000000,1.00000,1.000000,2.000000
25%,100930.500000,1329.500000,0.000000,NaN,2.000000,0.000000,39.000000,14.000000,12.000000,14.000000,...,7.000000,2.00000,2.000000,0.000000,0.554763,2.00000,9.000000,10.00000,9.000000,32.000000
50%,206466.000000,2455.500000,0.000000,NaN,3.000000,0.000000,84.000000,25.000000,20.000000,25.000000,...,13.000000,3.00000,3.000000,0.000000,0.706387,3.00000,16.000000,19.00000,16.000000,70.000000
75%,313024.500000,3910.000000,0.000000,NaN,4.000000,1.000000,163.000000,46.000000,31.000000,45.000000,...,24.000000,3.00000,3.000000,0.000000,0.903796,3.00000,32.250000,34.00000,32.000000,141.250000
max,424068.000000,4993.000000,1.000000,NaN,8.000000,2.000000,2470.000000,344.000000,161.000000,320.000000,...,105.000000,4.00000,3.000000,2.000000,1.475535,3.00000,966.000000,532.00000,966.000000,2270.000000


In [20]:
# 用户特征统计：对用户对点击、加购、购买、收藏等特征进行统计。
def col_cnt_(df_data, columns_list, action_type):
    try:
        data_dict = {}
        col_list = copy.deepcopy(columns_list)
        if action_type != None:
            col_list += ['action_type_path']
        for col in col_list:
            data_dict[col] = df_data[col].split(' ')
        path_len = len(data_dict[col])

        data_out = []
        for i_ in range(path_len):
            data_txt = ''
            for col_ in columns_list:
                if data_dict['action_type_path'][i_] == action_type:
                    data_txt += '_' + data_dict[col_][i_]
            # 此处与书上不一样
            if data_txt != '':
                data_out.append(data_txt)
            # if data_dict['action_type_path'][i_] == action_type:
            #     data_out.append('_'+data_dict['action_type_path'][i_])
        return len(data_out)
    except:
        return -1

def col_nunique_(df_data, columns_list, action_type):
    try:
        data_dict = {}
        col_list = copy.deepcopy(columns_list)
        if action_type != None:
            col_list += ['action_type_path']
        for col in col_list:
            data_dict[col] = df_data[col].split(' ')
        path_len = len(data_dict[col])

        data_out = []
        for i_ in range(path_len):
            data_txt = ''
            for col_ in columns_list:
                if data_dict['action_type_path'][i_] == action_type:
                    data_txt += '_' + data_dict[col_][i_]
            # 此处课本与书上不一样
            if data_txt != '':
                data_out.append(data_txt)
        return len(set(data_out))
    except:
        return -1

'''
columns_list: 要计算的列名
action_type：
'''
def user_col_cnt(df_data, columns_list, action_type, name):
    df_data[name] = df_data.apply(lambda x: col_cnt_(x, columns_list, action_type), axis=1)
    return df_data

def user_col_nunique(df_data, columns_list, action_type, name):
    df_data[name] = df_data.apply(lambda x: col_nunique_(x, columns_list, action_type), axis=1)
    return df_data

In [21]:
# 统计用户和店铺的关系：对店铺的用户点击次数、加购次数、购买次数、收藏次数等
# 点击次数
all_data_test = user_col_cnt(all_data_test, ['seller_path'], '0', 'user_cnt_0') # 该用户的点击次数
# 加购次数
all_data_test = user_col_cnt(all_data_test, ['seller_path'], '1', 'user_cnt_1') # 该用户的加购次数

In [22]:
all_data_test.tail()

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,...,seller_most_1,cat_most_1,brand_most_1,action_type_most_1,seller_most_1_cnt,cat_most_1_cnt,brand_most_1_cnt,action_type_most_1_cnt,user_cnt_0,user_cnt_1
1995,220293,2883,0.0,NaN,4.0,1.0,551995 497734 703172 468886 813265 1424 1424 1...,50 50 683 683 331 1401 1401 1401 1401 1401 737...,638 2782 2884 192 2982 2982 2982 2982 2982 298...,1444.0 84.0 7239.0 5638.0 5750.0 5750.0 5750.0...,...,2677,656,1579.0,0,17,14,17,41,41,0
1996,155013,3727,0.0,NaN,2.0,0.0,881928 833279 961513 305410 305410 305410 1057...,737 737 737 1349 1349 1349 737 737 737 737 737...,4390 3718 4993 702 702 702 3727 3727 3727 3727...,5223.0 3169.0 1865.0 1360.0 1360.0 1360.0 6816...,...,3727,737,6816.0,0,47,51,47,104,104,0
1997,24453,1487,0.0,NaN,5.0,1.0,1049364 1049364 282508 251929 291529 166817 45...,1075 1075 1213 1438 1200 407 1611 407 407 178 ...,3365 3365 1256 639 4016 4346 4346 4346 4346 43...,7524.0 7524.0 1573.0 1565.0 4358.0 2919.0 2919...,...,4346,407,2919.0,0,42,25,42,69,69,0
1998,155781,1861,0.0,NaN,0.0,2.0,437216 1040600 305379 168779 463220 895836 305...,407 407 1438 1438 178 1505 656 612 1553 184 18...,3163 118 4211 4685 4645 3910 816 4364 1910 203...,3881.0 6020.0 5242.0 7647.0 7837.0 5484.0 5784...,...,1199,1389,5409.0,0,5,32,5,55,55,0
1999,90501,3205,0.0,NaN,3.0,0.0,966016 420078 463793 102071 402374 463793 1517...,245 1656 720 720 125 720 1023 1023 177 177 177...,4600 4600 1071 3402 306 1071 206 344 2051 3155...,404.0 1307.0 2100.0 4564.0 7592.0 2100.0 2512....,...,1123,389,4029.0,0,11,7,11,40,40,0


In [23]:
# 特征组合
# 点击次数
all_data_test = user_col_cnt(all_data_test, ['seller_path', 'item_path'], '0', 'user_cnt_0')
# 不同店铺数
all_data_test = user_col_nunique(all_data_test, ['seller_path', 'item_path'], '0', 'seller_nunique_0')

In [24]:
all_data_test.describe()

,user_id,merchant_id,label,prob,age_range,gender,user_cnt,seller_nunique,cat_nunique,brand_nunique,...,time_stamp_min,time_stamp_std,time_stamp_range,seller_most_1_cnt,cat_most_1_cnt,brand_most_1_cnt,action_type_most_1_cnt,user_cnt_0,user_cnt_1,seller_nunique_0
count,2000.000000,2000.000000,2000.000000,0.0,1995.000000,1983.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000
mean,208239.475500,2533.319000,0.064500,NaN,3.135840,0.349975,140.082500,35.701500,23.987000,35.374000,...,0.004000,0.743902,2.57300,29.575000,30.53200,29.844500,123.778000,123.59300,0.133000,77.216000
std,122331.596829,1457.359826,0.245703,NaN,1.889127,0.530178,202.303623,35.853021,18.091365,35.298146,...,0.089376,0.264871,0.50279,46.845711,39.37676,47.037364,186.462639,186.53453,0.662216,110.667629
min,132.000000,10.000000,0.000000,NaN,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,...,0.000000,0.000000,0.00000,1.000000,1.00000,1.000000,2.000000,0.00000,0.000000,0.000000
25%,100930.500000,1329.500000,0.000000,NaN,2.000000,0.000000,39.000000,14.000000,12.000000,14.000000,...,0.000000,0.554763,2.00000,9.000000,10.00000,9.000000,32.000000,32.00000,0.000000,21.000000
50%,206466.000000,2455.500000,0.000000,NaN,3.000000,0.000000,84.000000,25.000000,20.000000,25.000000,...,0.000000,0.706387,3.00000,16.000000,19.00000,16.000000,70.000000,70.00000,0.000000,46.000000
75%,313024.500000,3910.000000,0.000000,NaN,4.000000,1.000000,163.000000,46.000000,31.000000,45.000000,...,0.000000,0.903796,3.00000,32.250000,34.00000,32.000000,141.250000,141.00000,0.000000,89.000000
max,424068.000000,4993.000000,1.000000,NaN,8.000000,2.000000,2470.000000,344.000000,161.000000,320.000000,...,2.000000,1.475535,3.00000,966.000000,532.00000,966.000000,2270.000000,2270.00000,12.000000,1446.000000


In [25]:
all_data_test.columns

Index(['user_id', 'merchant_id', 'label', 'prob', 'age_range', 'gender',
       'item_path', 'cat_path', 'seller_path', 'brand_path', 'time_stamp_path',
       'action_type_path', 'user_cnt', 'seller_nunique', 'cat_nunique',
       'brand_nunique', 'item_nunique', 'time_stamp_nunique',
       'action_type_nunique', 'time_stamp_max', 'time_stamp_min',
       'time_stamp_std', 'time_stamp_range', 'seller_most_1', 'cat_most_1',
       'brand_most_1', 'action_type_most_1', 'seller_most_1_cnt',
       'cat_most_1_cnt', 'brand_most_1_cnt', 'action_type_most_1_cnt',
       'user_cnt_0', 'user_cnt_1', 'seller_nunique_0'],
      dtype='object')

#### 3.3.7 利用CountVector 和 TF-IDF提取特征

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from scipy import sparse

tfidfVec = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS,
                           ngram_range=(1,1),
                           max_features=100)

columns_list = ['seller_path']
for i, col in enumerate(columns_list):
    tfidfVec.fit(all_data_test[col])
    data_ = tfidfVec.transform(all_data_test[col])
    if i == 0:
        data_cat = data_
    else:
        data_cat = sparse.hstack((data_cat, data_))

In [27]:
# 特征重命名和特征合并
df_tfidf = pd.DataFrame(data_cat.toarray())
df_tfidf.columns = ['tfidf_' + str(i) for i in df_tfidf.columns]
all_data_test = pd.concat([all_data_test, df_tfidf], axis=1)

In [28]:
all_data_test

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,0.0,0.0,0.012666,0.011178,0.0,0.0,0.0,0.0,0.012925,0.000000
1,34176,121,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,0.0,0.0,0.012666,0.011178,0.0,0.0,0.0,0.0,0.012925,0.000000
2,34176,4356,1.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,0.0,0.0,0.012666,0.011178,0.0,0.0,0.0,0.0,0.012925,0.000000
3,34176,2217,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,0.0,0.0,0.012666,0.011178,0.0,0.0,0.0,0.0,0.012925,0.000000
4,230784,4818,0.0,NaN,0.0,0.0,191923 191923 191923 191923 964906 229470 2294...,1023 1023 1023 1023 662 664 664 1544 664 662 6...,3545 3545 3545 3545 4566 2537 2537 2420 2537 4...,5860.0 5860.0 5860.0 5860.0 6322.0 6066.0 6066...,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,220293,2883,0.0,NaN,4.0,1.0,551995 497734 703172 468886 813265 1424 1424 1...,50 50 683 683 331 1401 1401 1401 1401 1401 737...,638 2782 2884 192 2982 2982 2982 2982 2982 298...,1444.0 84.0 7239.0 5638.0 5750.0 5750.0 5750.0...,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
1996,155013,3727,0.0,NaN,2.0,0.0,881928 833279 961513 305410 305410 305410 1057...,737 737 737 1349 1349 1349 737 737 737 737 737...,4390 3718 4993 702 702 702 3727 3727 3727 3727...,5223.0 3169.0 1865.0 1360.0 1360.0 1360.0 6816...,...,0.0,0.0,0.060478,0.000000,0.0,0.0,0.0,0.0,0.000000,0.834207
1997,24453,1487,0.0,NaN,5.0,1.0,1049364 1049364 282508 251929 291529 166817 45...,1075 1075 1213 1438 1200 407 1611 407 407 178 ...,3365 3365 1256 639 4016 4346 4346 4346 4346 43...,7524.0 7524.0 1573.0 1565.0 4358.0 2919.0 2919...,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
1998,155781,1861,0.0,NaN,0.0,2.0,437216 1040600 305379 168779 463220 895836 305...,407 407 1438 1438 178 1505 656 612 1553 184 18...,3163 118 4211 4685 4645 3910 816 4364 1910 203...,3881.0 6020.0 5242.0 7647.0 7837.0 5484.0 5784...,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


#### 3.3.8 嵌入特征

In [29]:
# pip install gensim==3.7.0
# notice: pip install smart-open==2.1
import gensim
# Train Word2Vec model

model = gensim.models.Word2Vec(
    all_data_test['seller_path'].apply(lambda x: x.split(' ')),
    size=100,
    window=5,
    min_count=5,
    workers=4
)
# model.save('product2vec.model')
# model = gensim.models.Word2Vec.load('product2vec.model')

def mean_w2v_(x, model, size=100):
    try:
        i = 0
        for word in x.split(' '):
            if word in model.wv.vocab:
                i += 1
                if i == 1:
                    vec = np.zeros(size)
                vec += model.wv[word]
        return vec / i
    except:
        return np.zeros(size)

def get_mean_w2v(df_data, columns, model, size):
    data_array = []
    for index, row in df_data.iterrows():
        w2v = mean_w2v_(row[columns], model, size)
        data_array.append(w2v)
    return pd.DataFrame(data_array)

df_embedding = get_mean_w2v(all_data_test, 'seller_path', model, 100)
df_embedding.columns = ['embedding_' + str(i) for i in df_embedding.columns]

In [30]:
# 嵌入特征和原始特征合并：
all_data_test = pd.concat([all_data_test, df_embedding], axis=1)
all_data_test

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,...,embedding_90,embedding_91,embedding_92,embedding_93,embedding_94,embedding_95,embedding_96,embedding_97,embedding_98,embedding_99
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,0.055015,0.204041,-0.830379,0.110582,0.930311,-0.532224,0.850213,-0.390218,0.243984,-0.015180
1,34176,121,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,0.055015,0.204041,-0.830379,0.110582,0.930311,-0.532224,0.850213,-0.390218,0.243984,-0.015180
2,34176,4356,1.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,0.055015,0.204041,-0.830379,0.110582,0.930311,-0.532224,0.850213,-0.390218,0.243984,-0.015180
3,34176,2217,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,0.055015,0.204041,-0.830379,0.110582,0.930311,-0.532224,0.850213,-0.390218,0.243984,-0.015180
4,230784,4818,0.0,NaN,0.0,0.0,191923 191923 191923 191923 964906 229470 2294...,1023 1023 1023 1023 662 664 664 1544 664 662 6...,3545 3545 3545 3545 4566 2537 2537 2420 2537 4...,5860.0 5860.0 5860.0 5860.0 6322.0 6066.0 6066...,...,-0.017308,-0.239950,-0.375473,0.356453,0.273118,-0.590683,0.138789,0.300676,-0.138416,0.107809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,220293,2883,0.0,NaN,4.0,1.0,551995 497734 703172 468886 813265 1424 1424 1...,50 50 683 683 331 1401 1401 1401 1401 1401 737...,638 2782 2884 192 2982 2982 2982 2982 2982 298...,1444.0 84.0 7239.0 5638.0 5750.0 5750.0 5750.0...,...,0.286826,0.269958,-0.860182,-1.501044,-0.005671,-0.120599,0.343620,-0.109789,1.056276,-0.021287
1996,155013,3727,0.0,NaN,2.0,0.0,881928 833279 961513 305410 305410 305410 1057...,737 737 737 1349 1349 1349 737 737 737 737 737...,4390 3718 4993 702 702 702 3727 3727 3727 3727...,5223.0 3169.0 1865.0 1360.0 1360.0 1360.0 6816...,...,-0.040684,-0.061386,-0.740309,-0.696075,0.432624,-0.548706,0.229110,-0.113541,-0.050330,0.163967
1997,24453,1487,0.0,NaN,5.0,1.0,1049364 1049364 282508 251929 291529 166817 45...,1075 1075 1213 1438 1200 407 1611 407 407 178 ...,3365 3365 1256 639 4016 4346 4346 4346 4346 43...,7524.0 7524.0 1573.0 1565.0 4358.0 2919.0 2919...,...,0.393848,-0.481799,-0.387822,0.008149,0.281056,-0.689266,-0.332687,-0.071592,-0.207583,-0.068026
1998,155781,1861,0.0,NaN,0.0,2.0,437216 1040600 305379 168779 463220 895836 305...,407 407 1438 1438 178 1505 656 612 1553 184 18...,3163 118 4211 4685 4645 3910 816 4364 1910 203...,3881.0 6020.0 5242.0 7647.0 7837.0 5484.0 5784...,...,-0.066527,-0.333414,-0.299121,-0.279565,0.046410,-0.247434,0.048280,-0.134018,0.014305,0.169057


#### 3.3.9 Stacking分类特征

In [31]:
# 1. Stacking特征工具包
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from scipy import sparse
import xgboost
import lightgbm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [32]:
# 2. 定义Stacking分类特征相关函数
def stacking_clf(clf, train_x, train_y, test_x, clf_name, kf, label_split=None):
    train = np.zeros((train_x.shape[0], 1))
    test = np.zeros((test_x.shape[0], 1))
    test_pre = np.empty((folds, test_x.shape[0], 1))
    cv_scores = []
    for i ,(train_index, test_index) in enumerate(kf.split(train_x, label_split)):
        tr_x = train_x[train_index]
        tr_y = train_y[train_index]
        te_x = train_x[test_index]
        te_y = train_y[test_index]

        if clf_name in ['rf', 'ada', 'gb', 'et', 'lr', 'knn', 'gnb']:
            clf.fit(tr_x, tr_y)
            pre = clf.predict(te_x)
            train[test_index] = pre[:,0].reshape(-1,1)
            test_pre[i, :] = clf.predict_proba(test_x)[:,0].reshape(-1, 1)
            cv_scores.append(log_loss(te_y, pre[:,0].reshape(-1,1)))
        elif clf_name in ['xgb']:
            train_matrix = clf.DMatrix(tr_x, label=tr_y, missing=-1)
            test_matrix = clf.DMatrix(te_x, label=te_y, missing=-1)
            # te_y -> train_y
            z = clf.DMatrix(test_x, label=train_y, missing=-1)
            params = {
                'booster': 'gbtree',
                'objective': 'multi:softprob',
                'eval_metric': 'mlogloss',
                'gamma': 1,
                'min_child_weight': 1.5,
                'max_depth': 5,
                'lambda': 10,
                'subsample': 0.7,
                'colsample_bytree': 0.7,
                'colsample_bylevel': 0.7,
                'eta': 0.03,
                'tree_method': 'exact',
                'seed': 2021,
                'num_class': 2
            }

            num_round = 10000
            early_stopping_rounds = 100
            watchlist = [(train_matrix, 'train'), (test_matrix, 'eval')]
            if test_matrix:
                model = clf.train(
                    params,
                    train_matrix,
                    num_boost_round=num_round,
                    evals=watchlist,
                    early_stopping_rounds=early_stopping_rounds
                )
                pre = model.predict(
                    test_matrix,
                    ntree_limit=model.best_ntree_limit
                )
                train[test_index] = pre[:,0].reshape(-1,1)
                test_pre[i, :] = model.predict(
                    z, ntree_limit=model.best_ntree_limit
                )[:,0].reshape(-1,1)
                cv_scores.append(log_loss(te_y, pre[:,0].reshape(-1,1)))
        elif clf_name in ['lgb']:
            train_matrix = clf.Dataset(tr_x, label=tr_y)
            test_matrix = clf.Dataset(te_x, label=te_y)
            params = {
                'boosting_type': 'gbdt',
                'objective': 'multiclass',
                'metric': 'multi_logloss',
                'min_child_weight': 1.5,
                'num_leaves': 2**5,
                'lambda_l2': 10,
                'subsample': 0.7,
                'colsample_bytree': 0.7,
                'colsample_bylevel': 0.7,
                'learning_rate': 0.03,
                'tree_method': 'exact',
                'seed': 2021,
                'num_class': 2,
                'silent': True
            }
            num_round = 10000
            early_stopping_rounds = 100
            # watchlist = [(train_matrix, 'train'), (test_matrix, 'eval')]
            if test_matrix:
                model = clf.train(
                    params,
                    train_matrix,
                    num_round,
                    valid_sets=test_matrix,
                    early_stopping_rounds=early_stopping_rounds
                )
                pre = model.predict(
                    te_x,
                    num_iteration=model.best_iteration
                )
                train[test_index] = pre[:,0].reshape(-1,1)
                test_pre[i, :] = model.predict(
                    test_x, num_iteration=model.best_iteration
                )[:,0].reshape(-1,1)
                cv_scores.append(mean_squared_error(te_y, pre[:,0].reshape(-1,1)))
        else:
            raise IOError('Please add new clf.')
        print('%s now score is:' %clf_name, cv_scores)
    test[:] = test_pre.mean(axis=0)
    print('%s_score_list:' %clf_name, cv_scores)
    print('%s_score_mean:' %clf_name, np.mean(cv_scores))
    return train.reshape(-1,1), test.reshape(-1,1)


def rf_clf(x_train, y_train, x_valid, kf, label_split=None):
    randomforest = RandomForestClassifier(
        n_estimators=1200,
        max_depth=20,
        n_jobs=-1,
        random_state=2021,
        max_features='auto',
        verbose=1
    )
    rf_train, rf_test = stacking_clf(
        randomforest,
        x_train,
        y_train,
        x_valid,
        'rf',
        kf,
        label_split=label_split
    )

    return rf_train, rf_test, 'rf'

def ada_clf(x_train, y_train, x_valid, kf, label_split=None):
    adaboost = AdaBoostClassifier(
        n_estimators=50,
        random_state=2021,
        learning_rate=0.01
    )
    ada_train, ada_test = stacking_clf(
        adaboost,
        x_train,
        y_train,
        x_valid,
        'ada',
        kf,
        label_split=label_split
    )

    return ada_train, ada_test, 'ada'

def gb_clf(x_train, y_train, x_valid, kf, label_split=None):
    gbdt = GradientBoostingClassifier(
        learning_rate=0.04,
        n_estimators=100,
        subsample=0.8,
        random_state=2021,
        max_depth=5,
        verbose=1
    )
    gbdt_train, gbdt_test = stacking_clf(
        gbdt,
        x_train,
        y_train,
        x_valid,
        'gb',
        kf,
        label_split=label_split
    )

    return gbdt_train, gbdt_test, 'gb'

def et_clf(x_train, y_train, x_valid, kf, label_split=None):
    extratree = ExtraTreesClassifier(
        n_estimators=1200,
        max_depth=35,
        n_jobs=-1,
        random_state=2021,
        max_features='auto',
        verbose=1
    )
    et_train, et_test = stacking_clf(
        extratree,
        x_train,
        y_train,
        x_valid,
        'et',
        kf,
        label_split=label_split
    )

    return et_train, et_test, 'et'

def xgb_clf(x_train, y_train, x_valid, kf, label_split=None):
    xgb_train, xgb_test = stacking_clf(
        xgboost,
        x_train,
        y_train,
        x_valid,
        'xgb',
        kf,
        label_split=label_split
    )

    return xgb_train, xgb_test, 'xgb'

def lgb_clf(x_train, y_train, x_valid, kf, label_split=None):
    lgb_train, lgb_test = stacking_clf(
        lightgbm,
        x_train,
        y_train,
        x_valid,
        'lgb',
        kf,
        label_split=label_split
    )

    return lgb_train, lgb_test, 'lgb'

def gnb_clf(x_train, y_train, x_valid, kf, label_split=None):
    gnb = GaussianNB()
    gnb_train, gnb_test = stacking_clf(
        gnb,
        x_train,
        y_train,
        x_valid,
        'gnb',
        kf,
        label_split=label_split
    )

    return gnb_train, gnb_test, 'gnb'

def lr_clf(x_train, y_train, x_valid, kf, label_split=None):
    logisticregression = LogisticRegression(
        n_jobs=-1,
        random_state=2021,
        C=0.1,
        max_iter=200
    )
    lr_train, lr_test = stacking_clf(
        logisticregression,
        x_train,
        y_train,
        x_valid,
        'lr',
        kf,
        label_split=label_split
    )

    return lr_train, lr_test, 'lr'

def knn_clf(x_train, y_train, x_valid, kf, label_split=None):
    kneighbors = KNeighborsClassifier(
        n_neighbors=200,
        n_jobs=-1,
    )
    knn_train, knn_test = stacking_clf(
        kneighbors,
        x_train,
        y_train,
        x_valid,
        'knn',
        kf,
        label_split=label_split
    )

    return knn_train, knn_test, 'knn'

In [33]:
# 3. 读取训练数据和验证数据
features_columns = [
    c for c in all_data_test.columns if c not in [
        'label','prob','seller_path','cat_path','brand_path','action_type_path','item_path','time_stamp_path'
    ]
]
x_train = all_data_test[
    ~all_data_test['label'].isna()][features_columns].values

y_train = all_data_test[
    ~all_data_test['label'].isna()]['label'].values

x_valid = all_data_test[
    all_data_test['label'].isna()][features_columns].values

In [34]:
def get_matrix(data):
    where_are_nan = np.isnan(data)
    where_are_inf = np.isinf(data)
    data[where_are_nan] = 0
    data[where_are_inf] = 0
    return data
x_train = np.float_(get_matrix(np.float_(x_train)))
y_train = np.int_(y_train)
x_valid = x_train

In [35]:
# 4. 使用lgb和xgb分类模型构造Stacking特征
from sklearn.model_selection import StratifiedKFold, KFold
folds = 5
seed = 1
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [37]:
# 选择lgb xgb作为基模型
clf_list = [lgb_clf, xgb_clf]
clf_list_col = ['lgb_clf', 'xgb_clf']

In [40]:
# 获取stacking特征
clf_list = clf_list
column_list = []
train_data_list = []
test_data_list = []
for clf in clf_list:
    train_data, test_data, clf_name = clf(x_train, y_train, x_valid, kf, label_split=None)
    train_data_list.append(train_data)
    test_data_list.append(test_data)
train_stacking = np.concatenate(train_data_list, axis=1)
test_stacking = np.concatenate(test_data_list, axis=1)


[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012961 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31308
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 223
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Info] Start training from score -0.066541
[LightGBM] [Info] Start training from score -2.743030
[1]	valid_0's multi_logloss: 0.240472
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[108]	valid_0's multi_logloss: 0.262658
[109]	valid_0's multi_logloss: 0.26267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[110]	valid_0's multi_logloss: 0.262792
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[111]	valid_0's multi_logloss: 0.263125
Early stopping, best iteration is:
[11]	valid_0's multi_logloss: 0.238901
lgb now score is: [0.824843928032179]
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006407 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total B

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	valid_0's multi_logloss: 0.318367
[76]	valid_0's multi_logloss: 0.3188
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[77]	valid_0's multi_logloss: 0.319428
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[78]	valid_0's multi_logloss: 0.319764
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[79]	valid_0's multi_logloss: 0.320302
[80]	valid_0's multi_logloss: 0.320802
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[81]	valid_0's multi_logloss: 0.321416
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[82]	valid_0's multi_logloss: 0.321782
[83]	valid_0's multi_logloss: 0.321948
[LightGBM] [Warning] No further splits with positiv

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[90]	valid_0's multi_logloss: 0.271555
[91]	valid_0's multi_logloss: 0.271756
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[92]	valid_0's multi_logloss: 0.272217
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[93]	valid_0's multi_logloss: 0.272351
[94]	valid_0's multi_logloss: 0.272749
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[95]	valid_0's multi_logloss: 0.272949
[96]	valid_0's multi_logloss: 0.272787
[97]	valid_0's multi_logloss: 0.272816
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[98]	valid_0's multi_logloss: 0.273045
[99]	valid_0's multi_logloss: 0.273479
[100]	valid_0's multi_logloss: 0.273626
[101]	valid_0's multi_logloss: 0.274036
[102]	valid_0's multi_logloss: 0.274156
[103]	valid_0's multi_logloss: 0.274201
[104]	valid_0's multi_logloss: 0.274444
Early stopping, best iteration is:


[15]	valid_0's multi_logloss: 0.210428
[16]	valid_0's multi_logloss: 0.210269
[17]	valid_0's multi_logloss: 0.209799
[18]	valid_0's multi_logloss: 0.209799
[19]	valid_0's multi_logloss: 0.210025
[20]	valid_0's multi_logloss: 0.209742
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[21]	valid_0's multi_logloss: 0.209798
[22]	valid_0's multi_logloss: 0.209542
[23]	valid_0's multi_logloss: 0.209571
[24]	valid_0's multi_logloss: 0.209402
[25]	valid_0's multi_logloss: 0.209196
[26]	valid_0's multi_logloss: 0.209008
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[27]	valid_0's multi_logloss: 0.209404
[28]	valid_0's multi_logloss: 0.209232
[29]	valid_0's multi_logloss: 0.20939
[30]	valid_0's multi_logloss: 0.209272
[31]	valid_0's multi_logloss: 0.209092
[32]	valid_0's multi_logloss: 0.20903
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

[139]	train-mlogloss:0.14729	eval-mlogloss:0.30191
[140]	train-mlogloss:0.14662	eval-mlogloss:0.30216
[141]	train-mlogloss:0.14600	eval-mlogloss:0.30240
[142]	train-mlogloss:0.14534	eval-mlogloss:0.30287
[143]	train-mlogloss:0.14472	eval-mlogloss:0.30296
[144]	train-mlogloss:0.14402	eval-mlogloss:0.30331
[145]	train-mlogloss:0.14343	eval-mlogloss:0.30358
[146]	train-mlogloss:0.14276	eval-mlogloss:0.30401
[147]	train-mlogloss:0.14222	eval-mlogloss:0.30415
[148]	train-mlogloss:0.14174	eval-mlogloss:0.30434
[149]	train-mlogloss:0.14123	eval-mlogloss:0.30455
[150]	train-mlogloss:0.14074	eval-mlogloss:0.30457
[151]	train-mlogloss:0.14024	eval-mlogloss:0.30465
[152]	train-mlogloss:0.13972	eval-mlogloss:0.30499
[153]	train-mlogloss:0.13918	eval-mlogloss:0.30521
[154]	train-mlogloss:0.13863	eval-mlogloss:0.30524
[155]	train-mlogloss:0.13811	eval-mlogloss:0.30537
[156]	train-mlogloss:0.13755	eval-mlogloss:0.30563
[157]	train-mlogloss:0.13708	eval-mlogloss:0.30608
[158]	train-mlogloss:0.13648	ev

[116]	train-mlogloss:0.17227	eval-mlogloss:0.25986
[117]	train-mlogloss:0.17133	eval-mlogloss:0.25996
[118]	train-mlogloss:0.17050	eval-mlogloss:0.26010
[119]	train-mlogloss:0.16973	eval-mlogloss:0.25993
[120]	train-mlogloss:0.16893	eval-mlogloss:0.26021
[121]	train-mlogloss:0.16819	eval-mlogloss:0.26026
[122]	train-mlogloss:0.16724	eval-mlogloss:0.26041
[123]	train-mlogloss:0.16653	eval-mlogloss:0.26082
[124]	train-mlogloss:0.16570	eval-mlogloss:0.26088
[125]	train-mlogloss:0.16506	eval-mlogloss:0.26113
[126]	train-mlogloss:0.16435	eval-mlogloss:0.26151
[127]	train-mlogloss:0.16364	eval-mlogloss:0.26165
[128]	train-mlogloss:0.16303	eval-mlogloss:0.26192
[129]	train-mlogloss:0.16234	eval-mlogloss:0.26189
[130]	train-mlogloss:0.16162	eval-mlogloss:0.26215
[131]	train-mlogloss:0.16087	eval-mlogloss:0.26237
[132]	train-mlogloss:0.16023	eval-mlogloss:0.26244
[133]	train-mlogloss:0.15952	eval-mlogloss:0.26240
[134]	train-mlogloss:0.15890	eval-mlogloss:0.26250
[135]	train-mlogloss:0.15820	ev

[62]	train-mlogloss:0.24431	eval-mlogloss:0.24291
[63]	train-mlogloss:0.24253	eval-mlogloss:0.24150
[64]	train-mlogloss:0.24089	eval-mlogloss:0.24017
[65]	train-mlogloss:0.23912	eval-mlogloss:0.23879
[66]	train-mlogloss:0.23751	eval-mlogloss:0.23773
[67]	train-mlogloss:0.23605	eval-mlogloss:0.23679
[68]	train-mlogloss:0.23458	eval-mlogloss:0.23549
[69]	train-mlogloss:0.23301	eval-mlogloss:0.23467
[70]	train-mlogloss:0.23135	eval-mlogloss:0.23337
[71]	train-mlogloss:0.23000	eval-mlogloss:0.23255
[72]	train-mlogloss:0.22853	eval-mlogloss:0.23157
[73]	train-mlogloss:0.22693	eval-mlogloss:0.23068
[74]	train-mlogloss:0.22568	eval-mlogloss:0.22985
[75]	train-mlogloss:0.22429	eval-mlogloss:0.22887
[76]	train-mlogloss:0.22306	eval-mlogloss:0.22803
[77]	train-mlogloss:0.22162	eval-mlogloss:0.22711
[78]	train-mlogloss:0.22024	eval-mlogloss:0.22626
[79]	train-mlogloss:0.21899	eval-mlogloss:0.22533
[80]	train-mlogloss:0.21773	eval-mlogloss:0.22480
[81]	train-mlogloss:0.21652	eval-mlogloss:0.22401


[224]	train-mlogloss:0.12296	eval-mlogloss:0.21823
[225]	train-mlogloss:0.12262	eval-mlogloss:0.21826
[226]	train-mlogloss:0.12215	eval-mlogloss:0.21842
[227]	train-mlogloss:0.12189	eval-mlogloss:0.21831
[228]	train-mlogloss:0.12160	eval-mlogloss:0.21848
[229]	train-mlogloss:0.12127	eval-mlogloss:0.21845
[230]	train-mlogloss:0.12090	eval-mlogloss:0.21859
xgb now score is: [2.3903688282426447, 2.2108744407258927, 2.5295380319934337, 2.5755847113672643]
[0]	train-mlogloss:0.67113	eval-mlogloss:0.67097
[1]	train-mlogloss:0.65048	eval-mlogloss:0.65014
[2]	train-mlogloss:0.63074	eval-mlogloss:0.63018
[3]	train-mlogloss:0.61192	eval-mlogloss:0.61090
[4]	train-mlogloss:0.59426	eval-mlogloss:0.59300
[5]	train-mlogloss:0.57740	eval-mlogloss:0.57599
[6]	train-mlogloss:0.56157	eval-mlogloss:0.55988
[7]	train-mlogloss:0.54653	eval-mlogloss:0.54460
[8]	train-mlogloss:0.53225	eval-mlogloss:0.53036
[9]	train-mlogloss:0.51865	eval-mlogloss:0.51644
[10]	train-mlogloss:0.50566	eval-mlogloss:0.50322
[11]

[154]	train-mlogloss:0.15430	eval-mlogloss:0.21530
[155]	train-mlogloss:0.15365	eval-mlogloss:0.21534
[156]	train-mlogloss:0.15300	eval-mlogloss:0.21532
[157]	train-mlogloss:0.15240	eval-mlogloss:0.21555
[158]	train-mlogloss:0.15180	eval-mlogloss:0.21570
[159]	train-mlogloss:0.15122	eval-mlogloss:0.21563
[160]	train-mlogloss:0.15071	eval-mlogloss:0.21561
[161]	train-mlogloss:0.15017	eval-mlogloss:0.21557
[162]	train-mlogloss:0.14963	eval-mlogloss:0.21538
[163]	train-mlogloss:0.14901	eval-mlogloss:0.21534
[164]	train-mlogloss:0.14840	eval-mlogloss:0.21554
[165]	train-mlogloss:0.14784	eval-mlogloss:0.21568
[166]	train-mlogloss:0.14728	eval-mlogloss:0.21559
[167]	train-mlogloss:0.14658	eval-mlogloss:0.21572
[168]	train-mlogloss:0.14603	eval-mlogloss:0.21574
[169]	train-mlogloss:0.14540	eval-mlogloss:0.21589
[170]	train-mlogloss:0.14489	eval-mlogloss:0.21581
[171]	train-mlogloss:0.14433	eval-mlogloss:0.21598
[172]	train-mlogloss:0.14383	eval-mlogloss:0.21586
[173]	train-mlogloss:0.14335	ev

In [41]:
# 5. 原始特征和Stacking特征合并
train = pd.DataFrame(np.concatenate([x_train, train_stacking], axis=1))
test = pd.DataFrame(np.concatenate([x_valid, test_stacking], axis=1))

In [42]:
# 特征重命名
df_train_all = pd.DataFrame(train)
df_train_all.columns = features_columns + clf_list_col
df_test_all = pd.DataFrame(test)
df_test_all.columns = features_columns + clf_list_col

In [43]:
# 获取数据ID及特征标签label：
df_train_all['user_id'] = all_data_test[~all_data_test['label'].isna()]['user_id']
df_test_all['user_id'] = all_data_test[~all_data_test['label'].isna()]['user_id']
df_train_all['label'] = all_data_test[~all_data_test['label'].isna()]['label']

In [45]:
df_train_all.describe()

,user_id,merchant_id,age_range,gender,user_cnt,seller_nunique,cat_nunique,brand_nunique,item_nunique,time_stamp_nunique,...,embedding_93,embedding_94,embedding_95,embedding_96,embedding_97,embedding_98,embedding_99,lgb_clf,xgb_clf,label
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,208239.475500,2533.319000,3.128000,0.347000,140.082500,35.701500,23.987000,35.374000,80.761000,17.845500,...,-0.127886,0.402131,-0.438481,0.166879,-0.238211,0.108946,0.052064,0.938600,0.922281,0.064500
std,122331.596829,1457.359826,1.893253,0.528896,202.303623,35.853021,18.091365,35.298146,112.293824,14.733505,...,0.331040,0.319369,0.271890,0.296283,0.281262,0.256775,0.277371,0.010250,0.041532,0.245703
min,132.000000,10.000000,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,...,-1.634824,-0.696722,-1.964599,-1.068851,-2.104269,-1.211202,-1.265249,0.831986,0.544951,0.000000
25%,100930.500000,1329.500000,2.000000,0.000000,39.000000,14.000000,12.000000,14.000000,24.000000,7.000000,...,-0.300196,0.174318,-0.573973,-0.021264,-0.355631,-0.031132,-0.085785,0.936934,0.915834,0.000000
50%,206466.000000,2455.500000,3.000000,0.000000,84.000000,25.000000,20.000000,25.000000,49.000000,13.000000,...,-0.101829,0.339616,-0.404014,0.148274,-0.202998,0.096808,0.068719,0.939995,0.931432,0.000000
75%,313024.500000,3910.000000,4.000000,1.000000,163.000000,46.000000,31.000000,45.000000,93.250000,24.000000,...,0.074427,0.576948,-0.266944,0.331435,-0.078693,0.243984,0.205505,0.942381,0.945251,0.000000
max,424068.000000,4993.000000,8.000000,2.000000,2470.000000,344.000000,161.000000,320.000000,1491.000000,105.000000,...,1.582394,2.183486,1.053383,1.657353,0.848559,1.339440,1.251514,0.962787,0.971894,1.000000


In [46]:
# 6. 保存特征
df_train_all.to_csv('./dataset/train_all.csv', header=True, index=False)
df_test_all.to_csv('./dataset/test_all.csv', header=True, index=False)